# CENG403 - Spring 2024 - THE1

In this take-home-exam, you will implement your own tensor library, called CerGen (short for CENG Gergen -- gergen: one of the Turkish translations of the term tensor).

Example usage:

```python
from cergen import rastgele_gercek,rastgele_dogal,cekirdek, gergen

boyut = ()
aralik = (0, 10)
g0 = rastgele_gercek(boyut, aralik)
print(g0)
0 boyutlu skaler gergen:
8

g1 = gergen([[1, 2, 3], [4, 5, 6]])
print(g1)
2x3 boyutlu gergen:
[[1, 2, 3]
 [4, 5, 6]]

g2 = gergen(rastgele_dogal((3, 1)))
print(g2)
3x1 boyutlu gergen
[[6],
[5],
[2]]

print((g1 * g2))


g3 = (g1 * (g2 + 3)).topla()

```


## 1 Task Description
In this homework, we introduce the gergen class, a custom data structure designed to provide a
hands-on experience with fundamental array operations, mirroring some functionalities typically
found in libraries like NumPy.

## Fundamental Operations:
Random number generation:

In [13]:
import random

def cekirdek(sayi: int):
    #Sets the seed for random number generation
    return random.seed(sayi)
def rastgele_dogal(boyut, aralik=(0,100), dagilim='uniform'):
    """
    Generates data of specified dimensions with random integer values and returns a gergen object.

    Parameters:
    boyut (tuple): Shape of the desired data.
    aralik (tuple, optional): (min, max) specifying the range of random values. Defaults to (0,100), which implies a default range.
    dagilim (string, optional): Distribution of random values ('uniform'). Defaults to 'uniform'.

    Returns:
    gergen: A new gergen object with random integer values.
    """
    if not isinstance(boyut, int) or boyut <= 0:
        raise ValueError("boyut must be a positive integer.")

    if not dagilim in ("normal", "uniform"):
        raise ValueError("dagilim must be 'normal' or 'uniform'.")

    # Called after checking validity
    random.seed()

    if dagilim == "normal":
        return [random.gauss(*aralik) for _ in range(boyut)]
    elif dagilim == "uniform":
        return [random.uniform(*aralik) for _ in range(boyut)]
    
    


def rastgele_gercek(boyut, aralik=(0.0, 1.0), dagilim='uniform'):
    """
    Generates a gergen of specified dimensions with random floating-point values.

    Parameters:
    boyut (tuple): Shape of the desired gergen.
    aralik (tuple, optional): (min, max) specifying the range of random values. Defaults to (0.0, 1.0) for uniform distribution.
    dagilim (string, optional): Distribution of random value ('uniform'). Defaults to 'uniform'.

    Returns:
    gergen: A new gergen object with random floating-point values.
    """
    
    # if not isinstance(boyut, (int, tuple)) or (isinstance(boyut, tuple) and not all(isinstance(dim, int) and dim > 0 for dim in boyut)):
    #     raise ValueError("boyut must be a positive integer or a tuple of positive integers.")

    if dagilim is not None and dagilim != "uniform":
        raise ValueError("dagilim must be None or 'uniform'.")

    if aralik[0] >= aralik[1]:
        raise ValueError("The lower bound of the range (aralik[0]) must be less than the upper bound (aralik[1]).")

    def rastgele_sayi():
        if dagilim == "uniform":
            return random.uniform(*aralik)
        else:
            raise ValueError("Unsupported distribution: " + dagilim)
        
    if isinstance(boyut, int):
        return [[rastgele_sayi()] for _ in range(boyut)]
    else:
        return [[rastgele_sayi() for _ in range(boyut[1])] for _ in range(boyut[0])]




Operation class implementation:

In [4]:
class Operation:
    def __call__(self, *operands):
        """
        Makes an instance of the Operation class callable.
        Stores operands and initializes outputs to None.
        Invokes the forward pass of the operation with given operands.

        Parameters:
            *operands: Variable length operand list.

        Returns:
            The result of the forward pass of the operation.
        """
        self.operands = operands
        self.outputs = None
        return self.ileri(*operands)

    def ileri(self, *operands):
        """
        Defines the forward pass of the operation.
        Must be implemented by subclasses to perform the actual operation.

        Parameters:
            *operands: Variable length operand list.

        Raises:
            NotImplementedError: If not overridden in a subclass.
        """
        raise NotImplementedError


In [8]:
import math
from typing import Union

class gergen:

    __veri = None #A nested list of numbers representing the data
    D = None # Transpose of data
    __boyut = None #Dimensions of the derivative (Shape)


    def __init__(self, veri=None):
    # The constructor for the 'gergen' class.
    #
    # This method initializes a new instance of a gergen object. The gergen can be
    # initialized with data if provided; otherwise, it defaults to None, representing
    # an empty tensor.
    #
    # Parameters:
    # veri (int/float, list, list of lists, optional): A nested list of numbers that represents the
    # gergen data. The outer list contains rows, and each inner list contains the
    # elements of each row. If 'veri' is None, the tensor is initialized without data.
    #
    # Example:
    # To create a tensor with data, pass a nested list:
    # tensor = gergen([[1, 2, 3], [4, 5, 6]])
    #
    # To create an empty tensor, simply instantiate the class without arguments:
    # empty_tensor = gergen()
    
        if veri is None:
            self.__veri = [] 
        elif isinstance(veri, list):
            # Check if it's a list of lists or a single list
            if all(isinstance(inner, list) for inner in veri):
                self.__veri = veri
            else:
                # If it's a single list, wrap it in another list to handle 1D Gergens
                self.__veri = [veri]
        else:
            raise TypeError("veri must be a list of lists or None.")
        
        self.__boyut()
        self.D = None

    def __boyut(self):
        if self.__veri is None:
            self.__boyut = None
        else:
            self.__boyut = tuple(len(row) for row in self.__veri)

    def __getitem__(self, index):
        if isinstance(self.__veri, (int, float)):  
            if isinstance(index, int) and index == 0:
                return self 
            else:
                raise IndexError("Single element gergen can only be accessed with index 0.")

        if isinstance(index, int): 
            if 0 <= index < len(self.__veri):
                return gergen(self.__veri[index])  
            else:
                raise IndexError("Index out of bounds for rows.")
        elif isinstance(index, tuple):
            if len(index) == len(self.__boyut): 
                if all(0 <= idx < dim for idx, dim in zip(index, self.__boyut)):
                    return self.__veri[index[0]][index[1]]
                else:
                    raise IndexError("Index out of bounds for elements.")
            else:
                raise ValueError("Invalid number of indices for multi-dimensional gergen.")
        else:
            raise TypeError("Index type must be int or tuple.")
        

    def __str__(self):
            if self.__veri is None:  
                return "Bos gergen"

            boyut_str = "x".join(str(dim) for dim in self.__boyut)  
            veri_str = ",\n ".join(str(row) for row in self.__veri) 
            return f"{boyut_str} boyutlu gergen :\n{veri_str}"

    def __mul__(self, other: Union['gergen', int, float]) -> 'gergen':
        """
        Multiplication operation for gergen objects.
        Called when a gergen object is multiplied by another, using the '*' operator.
        Could be element-wise multiplication or scalar multiplication, depending on the context.
        """
        if isinstance(other, gergen):  
            if self.__boyut != other.__boyut:
                raise ValueError("Dimensions must be equal for element-wise gergen multiplication.")
            return gergen([[self.__veri[i][j] * other.__veri[i][j] for j in range(self.__boyut[1])] for i in range(self.__boyut[0])])

        elif isinstance(other, (int, float)):  
            return gergen([[other * value for value in row] for row in self.__veri])

        else:
            raise TypeError("Can only multiply gergen with gergen or scalar (int/float).")

    def __truediv__(self, other: Union['gergen', int, float]) -> 'gergen':
        """
        Division operation for gergen objects.
        Called when a gergen object is divided by another, using the '/' operator.
        The operation is element-wise.
        """
        if not isinstance(other, (int, float)):
            raise TypeError("Can only divide gergen by a scalar value (int/float).")

        if other == 0:
            raise ZeroDivisionError("Cannot divide by zero.")

        return gergen([[float(value) / other for value in row] for row in self.__veri])


    def __add__(self, other: Union['gergen', int, float]) -> 'gergen':
        """
        Defines the addition operation for gergen objects.
        Called when a gergen object is added to another, using the '+' operator.
        The operation is element-wise.
        """
        if isinstance(other, gergen): 
            if self.__boyut != other.__boyut:
                raise ValueError("Dimensions must be equal for element-wise gergen addition.")
            return gergen([[self.__veri[i][j] + other.__veri[i][j] for j in range(self.__boyut[1])] for i in range(self.__boyut[0])])

        elif isinstance(other, (int, float)):  
            return gergen([[value + other for value in row] for row in self.__veri])

        else:
            raise TypeError("Can only add gergen with gergen or scalar (int/float).")
        
        
    def __radd__(self, other: Union[int, float]) -> 'gergen':
        """
        Called when a scalar (int/float) is added to a gergen object, using the '+' operator.
        The operation is element-wise.
        """
        return self.__add__(other)

    def __sub__(self, other: Union['gergen', int, float]) -> 'gergen':
        """
        Subtraction operation for gergen objects.
        Called when a gergen object is subtracted from another, using the '-' operator.
        The operation is element-wise.
        """
        if isinstance(other, gergen):  
            if self.__boyut != other.__boyut:
                raise ValueError("Dimensions must be equal for element-wise gergen subtraction.")
            return gergen([[self.__veri[i][j] - other.__veri[i][j] for j in range(self.__boyut[1])] for i in range(self.__boyut[0])])

        elif isinstance(other, (int, float)): 
            return gergen([[value - other for value in row] for row in self.__veri])

        else:
            raise TypeError("Can only subtract gergen with gergen or scalar (int/float).")
        
        
    def __rsub__(self, other: Union[int, float]) -> 'gergen':
        """
        Called when a scalar (int/float) is subtracted from a gergen object, using the '-' operator.
        The operation is element-wise.
        """
        # Subtract each element of the gergen from the scalar
        return gergen([[(other - value) for value in row] for row in self.__veri])
        

    def uzunluk(self):
    # Returns the total number of elements in the gergen
        if self.__veri is None:
            return 0
        return sum(len(row) for row in self.__veri)

    def boyut(self):
    # Returns the shape of the gergen
        if self.__veri is None:
            return None
        return tuple(len(row) for row in self.__veri)

    def devrik(self):
        #Returns the transpose of the gergen object as a new gergen object.

        if self.__veri is None:
            return None

        return gergen([[row[i] for row in self.__veri] for i in range(self.boyut[1])])

    def sin(self):
    #Calculates the sine of each element in the given `gergen`.
        if self.__veri is None:
            return None
        return gergen([[math.sin(value) for value in row] for row in self.__veri])

    def cos(self):
    #Calculates the cosine of each element in the given `gergen`.
        if self.__veri is None:
            return None
        return gergen([[math.cos(value) for value in row] for row in self.__veri])

    def tan(self):
    #Calculates the tangent of each element in the given `gergen`.
        if self.__veri is None:
            return None
        return gergen([[math.tan(value) for value in row] for row in self.__veri])

    def us(self, n: int):
    #Raises each element of the gergen object to the power 'n'. This is an element-wise operation.
        if not isinstance(n, int):
            raise ValueError("n must be an integer.")
        return gergen([[value**n for value in row] for row in self.__veri])
    pass

    def log(self):
        """
        Calculates the base-10 logarithm of each element in the gergen object and returns a new gergen with the results.

        Returns:
            gergen: A new gergen object containing the base-10 logarithm of each element.
        """

        if self.__veri is None:
            return None
        return gergen([[math.log10(value) for value in row] for row in self.__veri])

    def ln(self):
        """
        Calculates the natural logarithm of each element in the gergen object and returns a new gergen with the results.

        Returns:
            gergen: A new gergen object containing the natural logarithm of each element.
        """

        if self.__veri is None:
            return None
        return gergen([[math.log(value) for value in row] for row in self.__veri])

    def L1(self):
        """
        Calculates and returns the L1 norm of the gergen object.

        Returns:
            float: The L1 norm.
        """

        if self.__veri is None:
            return 0
        return sum(abs(value) for row in self.__veri for value in row)

    def L2(self):
        """
        Calculates and returns the L2 norm of the gergen object.

        Returns:
            float: The L2 norm.
        """

        if self.__veri is None:
            return 0
        return math.sqrt(sum(value**2 for row in self.__veri for value in row))

    def Lp(self, p: int):
        """
        Calculates and returns the Lp norm of the gergen object.
        """

        if not isinstance(p, int) or p <= 0:
            raise ValueError("p must be a positive integer.")

        if self.__veri is None:
            return 0

        sum_of_powers = sum(abs(value)**p for row in self.__veri for value in row)
        return sum_of_powers**(1/p)

    def listeye(self):
        """
        Converts the gergen object's data into a standard Python list structure.

        Returns:
            list: A list or nested list representing the gergen's data.
        """

        if self.__veri is None:
            return None
        return self.__veri.copy()

    def duzlestir(self):
        """
        Converts the gergen object's multi-dimensional structure into a 1D structure, effectively 'flattening' the object.

        Returns:
            gergen: A new gergen object with a 1D shape containing the flattened data.
        """

        if self.__veri is None:
            return None
        return gergen([value for row in self.__veri for value in row])

    def boyutlandir(self, yeni_boyut):
    #Reshapes the gergen object to a new shape 'yeni_boyut', which is specified as a tuple.
        if not isinstance(yeni_boyut, tuple):
            raise TypeError("yeni_boyut must be a tuple.")

        new_size = 1
        for dim in yeni_boyut:
            if not isinstance(dim, int) or dim <= 0:
                raise ValueError("Invalid dimensions in yeni_boyut. All dimensions must be positive integers.")
            new_size *= dim
        if new_size != self.uzunluk():
            raise ValueError("New shape is not compatible with the number of elements in the gergen.")

        reshaped_data = []
        current_row = []
        for value in self.__veri:
            current_row.append(value)
            if len(current_row) == yeni_boyut[1]:
                reshaped_data.append(current_row)
                current_row = []
        if current_row:
            reshaped_data.append(current_row)

        return gergen(reshaped_data)

    def ic_carpim(self, other):
    #Calculates the inner (dot) product of this gergen object with another.
        if not isinstance(self, gergen) or not isinstance(other, gergen):
            raise TypeError("Inner product can only be calculated between gergen objects.")

        if len(self.boyut) == 1 and len(other.boyut) == 1:
            if self.boyut[0] != other.boyut[0]:
                raise ValueError("Inner product of 1D gergens requires equal lengths.")
            return sum(value1 * value2 for value1, value2 in zip(self.__veri[0], other.__veri[0]))

        if len(self.boyut) == 2 and len(other.boyut) == 2:
            if self.boyut[1] != other.boyut[0]:
                raise ValueError("Incompatible dimensions for inner product (matrix multiplication).")
            product = [[0 for _ in range(other.boyut[1])] for _ in range(self.boyut[0])]
            for i in range(self.boyut[0]):
                for j in range(other.boyut[1]):
                    for k in range(self.boyut[1]):
                        product[i][j] += self.__veri[i][k] * other.__veri[k][j]
            return gergen(product)
        return gergen([[value1 * value2 for value1, value2 in zip(row1, row2)] for row1, row2 in zip(self.__veri, other.__veri)])

    def dis_carpim(self, other):
    #Calculates the outer product of this gergen object with another.
        if not isinstance(self, gergen) or not isinstance(other, gergen):
            raise TypeError("Outer product can only be calculated between gergen objects.")

        if len(self.boyut) != 1 or len(other.boyut) != 1:
            raise ValueError("Outer product can only be calculated between 1D gergens (vectors).")

        product = [[value1 * value2 for value2 in other.__veri] for value1 in self.__veri]
        return gergen(product)
    
    def topla(self, eksen=None):
    #Sums up the elements of the gergen object, optionally along a specified axis 'eksen'.
        if not (isinstance(eksen, int) or eksen is None):
            raise TypeError("eksen must be an integer or None.")

        if eksen is not None and (eksen < 0 or eksen >= len(self.boyut)):
            raise ValueError("eksen is out of bounds for the gergen's dimensions.")

        if eksen is None:
            return sum(value for row in self.__veri for value in row)

        if eksen == 0:  
            return gergen([sum(row[i] for row in self.__veri) for i in range(self.boyut[1])])
        elif eksen == 1:  
            return gergen([sum(row) for row in self.__veri])
        else:
            raise ValueError("Unsupported axis for summation.")
        
    def ortalama(self, eksen=None):
    #Calculates the average of the elements of the gergen object, optionally along a specified axis 'eksen'.
        if not (isinstance(eksen, int) or eksen is None):
            raise TypeError("eksen must be an integer or None.")

        if eksen is not None and (eksen < 0 or eksen >= len(self.boyut)):
            raise ValueError("eksen is out of bounds for the gergen's dimensions.")

        if eksen is None:
            return sum(value for row in self.__veri for value in row) / self.uzunluk()

        if eksen == 0:  # Column-wise average
            return gergen([sum(row[i] for row in self.__veri) / len(self.__veri) for i in range(self.boyut[1])])
        elif eksen == 1:  # Row-wise average
            return gergen([sum(row) / len(row) for row in self.__veri])
        else:
            raise ValueError("Unsupported axis for averaging.")

## 2 Compare with NumPy

In [10]:
import numpy as np              # NumPy, for working with arrays/tensors
import time                     # For measuring time


**Example 1:**
Using rastgele_gercek(), generate two gergen objects with shapes (64,64) and calculate the a.ic carpim(b). Then, calculate the same function with NumPy and report the time and difference.

In [45]:
def example_1():
    #Example 1
    boyut = (64,64)
    g1 = rastgele_gercek(boyut)
    g2 = rastgele_gercek(boyut)

    start = time.time()
    #TODO
    #Apply given equation
    end = time.time()

    start_np = time.time()
    #Apply the same equation for NumPy equivalent
    end_np = time.time()

    #TODO:
    #Compare if the two results are the same
    #Report the time difference
    print("Time taken for gergen:", end-start)
    print("Time taken for numpy:", end_np-start_np)


**Example 2:**
Using rastgele_gercek(), generate three gergen’s a, b and c with shapes (4,16,16,16). Calculate given equation:

> (a×b + a×c + b×c).ortalama()

Report the time and whether there exists any computational difference in result with their NumPy equivalent.

In [53]:
def example_2():
    
    # Generate random data using rastgele_gercek()
    a = np.array(rastgele_gercek((4, 16, 16, 16)))
    b = np.array(rastgele_gercek((4, 16, 16, 16)))
    c = np.array(rastgele_gercek((4, 16, 16, 16)))
    
    # Calculate using given equation
    start_time = time.time()
    resultOfMyCode = (a * b + a * c + b * c).mean()
    end_time = time.time()
    print("Time taken using custom function:", end_time - start_time)
    print("Result using custom function:", resultOfMyCode)
    
    # Generate random data using NumPy directly
    np_a = np.random.rand(4, 16, 16, 16)
    np_b = np.random.rand(4, 16, 16, 16)
    np_c = np.random.rand(4, 16, 16, 16)
    
    # Calculate using NumPy
    start_time = time.time()
    resultOfNumpyCode = (np_a * np_b + np_a * np_c + np_b * np_c).mean()
    end_time = time.time()
    print("Time taken using NumPy:", end_time - start_time)
    print("Result using NumPy:", resultOfNumpyCode)


example_2()

Time taken using custom function: 5.221366882324219e-05
Result using custom function: 0.6641184520307781
Time taken using NumPy: 0.0003342628479003906
Result using NumPy: 0.7537756376359274


**Example 3**: Using rastgele_gercek(), generate three gergen’s a and b with shapes (3,64,64). Calculate given equation:


> $\frac{\ln\left(\left(\sin(a) + \cos(b)\right)^2\right)}{8}$


Report the time and whether there exists any computational difference in result with their NumPy equivalent.


In [47]:
def example_3():
    #Example 3
    #TODO:
    pass